In [1]:
# Don't run on actual code
import os
os.chdir('../')

In [2]:
from fairlearn.metrics import MetricFrame, selection_rate, false_positive_rate, true_positive_rate
from fairlearn.metrics import demographic_parity_difference, demographic_parity_ratio, equalized_odds_difference
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds
import sys
sys.path.append("./subjects/")
import LogisticRegression
import Decision_Tree_Classifier
import TreeRegressor
import Discriminant_Analysis
import SVM

from adf_utils.config import census, credit, bank, compas
from adf_data.census import census_data
from adf_data.credit import credit_data
from adf_data.bank import bank_data
from adf_data.compas import compas_data

import subprocess
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
def getMostAccurateConfig(model,dataset,sensitive_feature,algo):
    df = pd.read_csv("./Dataset" + "/" + f"{model}_{dataset}_{sensitive_feature}_{algo}.csv")
    return df[df['score'] == df['score'].max()].iloc[0].inp

['/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/home/nxy5065/.local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/home/nxy5065/.local/lib/python3.6/site-packages/IPython/extensions', '/home/nxy5065/.ipython', './subjects/', '../', '../']


In [3]:
models = [("LR", "LogisticRegression"),("RF","TreeRegressor"),("SV","SVM"),("DT","Decision_Tree_Classifier")]
datasets = [("census", "gender",9), ("census", "race",8), ("credit", "gender",9), ("bank","age",1), ("compas","gender",1), ("compas","race",3)]

In [4]:
equalized_moment = EqualizedOdds(difference_bound=.1)

In [5]:
def get_data(dataset):
    data = {"census":census_data, "credit":credit_data, "bank":bank_data, "compas": compas_data}
    data_config = {"census":census, "credit":credit, "bank":bank, "compas": compas}
    X, Y, input_shape, nb_classes = data[dataset]()
    Y = np.argmax(Y, axis=1)
    # X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)
    return train_test_split(X, Y, random_state=0)

In [6]:
for m in models:
    for d in datasets:
        most_accurate = getMostAccurateConfig(m[0],d[0],d[1],"mutation")
        if(m[1] == "LogisticRegression"):
            input_program = input_program = LogisticRegression.logistic_regression
        elif(m[1] == "Decision_Tree_Classifier"):
            input_program = Decision_Tree_Classifier.DecisionTree
        elif(m[1] == "TreeRegressor"):
            input_program = TreeRegressor.TreeRegress
        elif(m[1] == "SVM"):
            input_program = SVM.SVM
        X_train, X_test, y_train, y_test = get_data(d[0])
        
        # I only really need to use the parser for the most accurate 'inp'
        # Exponentiated Gradient will retrain everything so the fit that this does will be "wasted"
        _, model, _, _, _, _ = input_program(most_accurate,X_train, X_test, y_train, y_test, d[2])
        training_model = ExponentiatedGradient(model,equalized_moment)
        training_model.fit(X_train,y_train,sensitive_features=X_train[:,d[2]-1])
        

[2, 1, 1, 0.19114388044017835, 7.508981760100949, 1, 3.7561736113233772, 976, 2, 1, 0, 0, -3, 1, 1]
['lbfgs', 'elasticnet', False, 0.0001, 1.0, False, 1.0, 950, 'ovr', 'none', 'None', 'None', 1, True, 'int', '1', 'int', '4', 'int', '3', 'int', '8', 'int', '8', 'int', '0', 'int', '4', 'int', '4', 'int', '0', 'int', '1', 'int', '7', 'int', '8', 'int', '3', 'int', '5', 'int', ',', 'int', ' ', 'int', '7', 'int', '.', 'int', '5', 'int', '0', 'int', '8', 'int', '9', 'int', '8', 'int', '1', 'int', '7', 'int', '6', 'int', '0', 'int', '1', 'int', '0', 'int', '0', 'int', '9', 'int', '4', 'int', '9', 'int', ',', 'int', ' ', 'int', '1', 'int', ',', 'int', ' ', 'int', '3', 'int', '.', 'int', '7', 'int', '5', 'int', '6', 'int', '1', 'int', '7', 'int', '3', 'int', '6', 'int', '1', 'int', '1', 'int', '3', 'int', '2', 'int', '3', 'int', '3', 'int', '7', 'int', '7', 'int', '2', 'int', ',', 'int', ' ', 'int', '9', 'int', '7', 'int', '6', 'int', ',', 'int', ' ', 'int', '2', 'int', ',', 'int', ' ', 'int', 

AssertionError: data can be loaded only once